# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats.contingency import association
import statsmodels.api as sm 
import scipy.stats as stats


%matplotlib inline
pd.set_option('display.max_columns', None)

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
pokemon = pd.read_csv('pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
pokemon.head(5)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [4]:
pokemon["Legendary"].value_counts()

Legendary
False    735
True      65
Name: count, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [5]:
pokemoncopy = pokemon.copy()

In [6]:
df = pokemoncopy.groupby("Legendary")["Total"].agg(["mean", "std"])
df

,mean,std
Legendary,,
False,417.213605,106.760417
True,637.384615,60.937389


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

H0: totallegendary = totalnolegendary

In [7]:
alpha = 0.05

In [8]:
df1 = pokemoncopy[pokemoncopy["Legendary"] == True]["Total"]
df2 = pokemoncopy[pokemoncopy["Legendary"] == False]["Total"]
df1 = pd.DataFrame(df1)
df2 = pd.DataFrame(df2)

In [9]:
df1

,Total
156,580
157,580
158,580
162,680
163,780
...,...
795,600
796,700
797,600
798,680


In [10]:
df2

,Total
0,318
1,405
2,525
3,625
4,309
...,...
787,494
788,304
789,514
790,245


In [11]:
df1.var()

Total    3713.365385
dtype: float64

In [12]:
df2.var()

Total    11397.786736
dtype: float64

In [13]:
stats.ttest_ind(df1, df2, equal_var=False, )

TtestResult(statistic=array([25.83357439]), pvalue=array([9.35795434e-47]), df=array([102.79988763]))

In [14]:
p_value = 9.35795434e-47

In [15]:
stats.ttest_ind(df1, df2, equal_var=False, nan_policy="omit")

TtestResult(statistic=array([25.83357439]), pvalue=array([9.35795434e-47]), df=array([102.79988763]))

What do you conclude from this test? Write your conclusions below.

In [16]:
if p_value > alpha:
    print("We are not able to reject the null hypothesis")
else:
    print("We reject the null hypotesis")

We reject the null hypotesis


How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [17]:
pokemoncopy["Type 1"].value_counts()

Type 1
Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: count, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [18]:
df3 = pokemoncopy.groupby("Type 1")["Total"].agg(["mean", "std"])
df3

,mean,std
Type 1,,
Bug,378.927536,117.875223
Dark,445.741935,109.126217
Dragon,550.531250,146.267538
Electric,443.409091,105.721952
Fairy,413.176471,123.781680
Fighting,416.444444,102.464378
Fire,458.076923,109.760496
Flying,485.000000,161.400124
Ghost,439.562500,110.072685


In [19]:
df3 = df3.reset_index()
df3

,Type 1,mean,std
0,Bug,378.927536,117.875223
1,Dark,445.741935,109.126217
2,Dragon,550.531250,146.267538
3,Electric,443.409091,105.721952
4,Fairy,413.176471,123.781680
5,Fighting,416.444444,102.464378
6,Fire,458.076923,109.760496
7,Flying,485.000000,161.400124
8,Ghost,439.562500,110.072685
9,Grass,421.142857,106.650626


In [20]:
df3["mean_normalize_by_water"] = df3["mean"] / df3["mean"][17]
df3["std_normalize_by_water"] = df3["std"] / df3["std"][17]

In [21]:
df3

,Type 1,mean,std,mean_normalize_by_water,std_normalize_by_water
0,Bug,378.927536,117.875223,0.880295,1.041409
1,Dark,445.741935,109.126217,1.035513,0.964112
2,Dragon,550.531250,146.267538,1.278951,1.292250
3,Electric,443.409091,105.721952,1.030093,0.934036
4,Fairy,413.176471,123.781680,0.959859,1.093591
5,Fighting,416.444444,102.464378,0.967451,0.905256
6,Fire,458.076923,109.760496,1.064168,0.969716
7,Flying,485.000000,161.400124,1.126714,1.425944
8,Ghost,439.562500,110.072685,1.021157,0.972474
9,Grass,421.142857,106.650626,0.978366,0.942241


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

H0: meanWater = meanNoWater

In [49]:
dfwater = pokemoncopy[pokemoncopy["Type 1"] == "Water"]["Total"]
dfnowater = pokemoncopy[pokemoncopy["Type 1"] != "Water"]["Total"]

In [51]:
dfwater

9      314
10     405
11     530
12     630
59     320
      ... 
724    314
725    405
726    530
762    330
763    500
Name: Total, Length: 112, dtype: int64

In [52]:
dfnowater

0      318
1      405
2      525
3      625
4      309
      ... 
795    600
796    700
797    600
798    680
799    600
Name: Total, Length: 688, dtype: int64

In [53]:
stats.ttest_ind(dfwater, dfnowater, equal_var=True)

TtestResult(statistic=-0.4418547448849676, pvalue=0.6587140317488793, df=798.0)

In [54]:
p_value = 0.6587140317488793

Write your conclusion below.

In [55]:
if p_value > alpha:
    print("We are not able to reject the null hypothesis")
else:
    print("We reject the null hypotesis")

We are not able to reject the null hypothesis


# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [47]:
pokemoncopy.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


H0: Attack=Defense

In [58]:
stats.ttest_rel(pokemoncopy["Attack"], pokemoncopy["Defense"])

TtestResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05, df=799)

In [59]:
p_value = 1.7140303479358558e-05

In [60]:
if p_value > alpha:
    print("We are not able to reject the null hypothesis")
else:
    print("We reject the null hypotesis")

We reject the null hypotesis


Describe the results of the test in the cell below.

no hay relacion alguna

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [25]:
H0: SpecialAttack=SpecialDefense

In [62]:
stats.ttest_rel(pokemoncopy["Sp. Atk"], pokemoncopy["Sp. Def"])

TtestResult(statistic=0.853986188453353, pvalue=0.3933685997548122, df=799)

In [63]:
p_value = 0.3933685997548122

In [64]:
if p_value > alpha:
    print("We are not able to reject the null hypothesis")
else:
    print("We reject the null hypotesis")

We are not able to reject the null hypothesis


Describe the results of the test in the cell below.

In [26]:
si que eparece que hay una relacion entre estas dos estadisticas

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

H0: Attack - Defense

In [65]:
df5 = pokemoncopy["Attack"] - pokemoncopy["Defense"]

In [67]:
stats.ttest_1samp(df5, 0)

TtestResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05, df=799)

In [68]:
p_value = 1.7140303479358558e-05

In [69]:
if p_value > alpha:
    print("We are not able to reject the null hypothesis")
else:
    print("We reject the null hypotesis")

We reject the null hypotesis


# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [71]:
df6 = pd.crosstab(pokemoncopy["Legendary"], pokemoncopy["Type 1"] == "Water")

In [72]:
df6

Type 1,False,True
Legendary,,
False,627,108
True,61,4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [76]:
stats.chi2_contingency(df6, correction=False, lambda_=0.05)

Chi2ContingencyResult(statistic=4.3211005504280084, pvalue=0.03764256909125677, dof=1, expected_freq=array([[632.1, 102.9],
       [ 55.9,   9.1]]))

Based on a 95% confidence, should we reject the null hypothesis?

### pongo False xq tengu una frecuencia que no es baja por lo que no quiero una sobreestimacion de chi2


In [77]:
p_value = 0.03764256909125677


In [78]:
if p_value > alpha:
    print("We are not able to reject the null hypothesis")
else:
    print("We reject the null hypotesis")

We reject the null hypotesis
